In [2]:
!pip install torch==2.2.0 torchtext==0.17.0


In [3]:
from torchtext.vocab import build_vocab_from_iterator
import torch# si tu as écrit ça



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/amirbenziane/.conda/envs/nlpe/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/amirbenziane/.conda/envs/nlpe/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/amirbenziane/.conda/envs/nlpe/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/amirbenziane/.conda/envs/nlpe/lib/python3.10/site-packages/traitlets/config/application.py",

In [4]:
def load_file(path):
    texts = []
    emotions = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                text, label = line.split(";")
                texts.append(text)
                emotions.append(label)
    return texts, emotions


In [5]:
text, emotion = load_file("dataset/train.txt")

print(text[2])
#print(tokenizer(text[2]))
print(emotion[2])



im grabbing a minute to post i feel greedy wrong
anger


In [6]:
def tokenizer(phrase):
    texts = phrase.split()
    return texts
print(tokenizer(text[1]))

['i', 'can', 'go', 'from', 'feeling', 'so', 'hopeless', 'to', 'so', 'damned', 'hopeful', 'just', 'from', 'being', 'around', 'someone', 'who', 'cares', 'and', 'is', 'awake']


In [7]:
def tokenizer_list(phrases):
    tokens = []
    for phrase in phrases:
        tokens.extend(tokenizer(phrase))
    return tokens
print(tokenizer_list(text)[5])
lst = tokenizer_list(text)

can


In [8]:
#build_vocab_from_iterator a faire ici
from torchtext.vocab import build_vocab_from_iterator
vocab ={"<pad>":0, "<unk>":1}
i = 2
for token in lst:
    if not token in vocab:
        vocab[token] = i
        i += 1

class MyVocab(dict):
    def __init__(self, mapping, unk_token="<unk>"):
        super().__init__(mapping)   # on garde le dict classique
        self.unk_token = unk_token
        self.unk_idx = mapping[unk_token]

    def __getitem__(self, token):
        # Si le mot existe → retourne l’index
        # Sinon → retourne l’index de <unk>
        return super().get(token, self.unk_idx)
    def copy(self):
        """
        Retourne une copie indépendante de l'objet MyVocab
        avec les mêmes données et attributs.
        """
        return MyVocab(dict(self), unk_token=self.unk_token)

vocab = MyVocab(vocab, unk_token="<unk>")
#vocab = build_vocab_from_iterator(lst,specials=["<pad>", "<unk>"])
print(vocab["i"])


print(vocab["caca"])

2
1


In [9]:
print(emotion)

['sadness', 'sadness', 'anger', 'love', 'anger', 'sadness', 'surprise', 'fear', 'joy', 'love', 'sadness', 'joy', 'anger', 'sadness', 'joy', 'joy', 'sadness', 'sadness', 'sadness', 'fear', 'anger', 'fear', 'joy', 'joy', 'anger', 'sadness', 'sadness', 'sadness', 'anger', 'joy', 'joy', 'fear', 'surprise', 'anger', 'joy', 'joy', 'joy', 'joy', 'anger', 'joy', 'joy', 'joy', 'joy', 'joy', 'sadness', 'sadness', 'joy', 'love', 'joy', 'anger', 'joy', 'sadness', 'anger', 'fear', 'joy', 'sadness', 'sadness', 'surprise', 'joy', 'joy', 'joy', 'love', 'fear', 'fear', 'surprise', 'anger', 'anger', 'sadness', 'love', 'joy', 'sadness', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'joy', 'joy', 'joy', 'anger', 'sadness', 'anger', 'anger', 'anger', 'joy', 'joy', 'joy', 'joy', 'sadness', 'fear', 'love', 'anger', 'sadness', 'anger', 'love', 'sadness', 'joy', 'joy', 'sadness', 'anger', 'love', 'joy', 'joy', 'joy', 'sadness', 'joy', 'joy', 'joy', 'joy', 'sadness', 'joy', 'joy', 'love', 'sadness', 'sadne

In [10]:
type_emotion = build_vocab_from_iterator([emotion])
print(type_emotion["sadness"])

1


In [11]:
code_phrases = []
def encode_function(code_phrasess):
    i = 0
    for phrase in text:
        code_phrase = []
        for texte in tokenizer(phrase):
            code_phrase.append(vocab[texte])
            i=i+1
        code_phrasess.append(code_phrase)


encode_function(code_phrases)
print(code_phrases[8])


[2, 76, 46, 77, 78, 79, 80, 2, 4, 39, 78, 81, 82, 83, 21, 84, 26, 85, 86]


In [12]:

print(len(code_phrases))
print(len(text))


16000
16000


In [13]:
max_len = max(len(seq) for seq in code_phrases)
tensor_code_phrases =torch.LongTensor([
        seq + [0] * (max_len - len(seq)) for seq in code_phrases
    ])
print(tensor_code_phrases.shape)

torch.Size([16000, 66])


In [14]:
from collections import Counter
all_tokens = tensor_code_phrases.view(-1).tolist()

# Compter la fréquence de chaque token
counter = Counter(all_tokens)


if 0 in counter:
    del counter[0]
print(len(vocab))
sum=0
for (key, value) in counter.items():
    sum+=value
ratio = sum/len(vocab)
print(ratio)
import pandas as pd

df = pd.Series(counter.values())

print(df.describe(percentiles=[.2, .8]))
#exemple: {0: 8, 2: 4, 31: 1, ...}

15214
20.156500591560405
count    15212.000000
mean        20.159151
std        294.261709
min          1.000000
20%          1.000000
50%          1.000000
80%          6.000000
max      25859.000000
dtype: float64


In [15]:
print(counter)

Counter({2: 25859, 4: 11183, 21: 9589, 12: 8972, 35: 8370, 26: 6200, 9: 5112, 39: 5112, 102: 4990, 179: 4283, 123: 3433, 40: 3127, 66: 2908, 10: 2527, 79: 2431, 24: 2430, 176: 2309, 64: 2255, 53: 2227, 76: 2224, 22: 2184, 133: 2088, 31: 2082, 77: 2015, 136: 1827, 34: 1795, 305: 1778, 68: 1565, 42: 1551, 203: 1471, 15: 1391, 213: 1376, 236: 1367, 55: 1246, 214: 1174, 209: 1157, 257: 1123, 95: 1066, 6: 966, 141: 942, 207: 935, 299: 897, 387: 896, 282: 886, 376: 884, 38: 853, 46: 853, 324: 809, 181: 808, 348: 805, 108: 794, 409: 774, 322: 773, 262: 740, 47: 736, 149: 719, 156: 705, 37: 697, 8: 689, 16: 681, 506: 673, 235: 654, 335: 651, 304: 646, 237: 645, 328: 644, 252: 634, 100: 619, 105: 596, 512: 596, 41: 595, 45: 587, 219: 575, 798: 568, 19: 567, 127: 567, 70: 551, 370: 537, 89: 523, 554: 523, 386: 518, 336: 516, 94: 513, 216: 500, 289: 496, 751: 487, 210: 482, 190: 480, 106: 474, 395: 470, 515: 453, 91: 445, 88: 444, 283: 443, 125: 442, 298: 428, 192: 419, 81: 411, 547: 403, 7: 393,

In [16]:
#Nettoyage de la base
vocab_clean = vocab.copy()
vocab_clean = {
    k: v for k, v in vocab.items()
    if (counter[v] >= 6) or (k in ["<pad>", "<unk>"])
}
vocab_clean =MyVocab(vocab_clean, unk_token="<unk>")



#Sortir les phrases
code_phrases_clean = []
def encode_function_clean(code_phrasess):
    for phrase in text:
        code_phrase = []
        for texte in tokenizer(phrase):
            code_phrase.append(vocab_clean[texte])
        code_phrasess.append(code_phrase)


print(len(vocab_clean))

3060


In [17]:
import torch
from torch.utils.data import DataLoader

import torch.nn.functional as F
def rebuild_vocab(vocab_clean, unk_token="<unk>"):
    new_mapping = {word: idx for idx, word in enumerate(vocab_clean.keys())}
    return MyVocab(new_mapping, unk_token=unk_token)

vocab_clean = rebuild_vocab(vocab_clean, unk_token="<unk>")
encode_function_clean(code_phrases_clean)

print(vocab_clean)
tensor_code_phrases_clean =torch.LongTensor([
        seq + [0] * (max_len - len(seq)) for seq in code_phrases_clean
    ])
# Créer un DataLoader
loader = DataLoader(tensor_code_phrases_clean, batch_size=20, shuffle=True)
print(len(vocab_clean))

{'<pad>': 0, '<unk>': 1, 'i': 2, 'didnt': 3, 'feel': 4, 'humiliated': 5, 'can': 6, 'go': 7, 'from': 8, 'feeling': 9, 'so': 10, 'hopeless': 11, 'to': 12, 'hopeful': 13, 'just': 14, 'being': 15, 'around': 16, 'someone': 17, 'who': 18, 'cares': 19, 'and': 20, 'is': 21, 'awake': 22, 'im': 23, 'a': 24, 'minute': 25, 'post': 26, 'greedy': 27, 'wrong': 28, 'am': 29, 'ever': 30, 'nostalgic': 31, 'about': 32, 'the': 33, 'will': 34, 'know': 35, 'that': 36, 'it': 37, 'still': 38, 'on': 39, 'grouchy': 40, 'ive': 41, 'been': 42, 'little': 43, 'burdened': 44, 'lately': 45, 'wasnt': 46, 'sure': 47, 'why': 48, 'was': 49, 'taking': 50, 'or': 51, 'times': 52, 'amount': 53, 'fallen': 54, 'asleep': 55, 'lot': 56, 'faster': 57, 'but': 58, 'also': 59, 'like': 60, 'funny': 61, 'as': 62, 'confused': 63, 'life': 64, 'teenager': 65, 'jaded': 66, 'year': 67, 'old': 68, 'man': 69, 'have': 70, 'with': 71, 'for': 72, 'years': 73, 'has': 74, 'well': 75, 'made': 76, 'huge': 77, 'romantic': 78, 'too': 79, 'make': 80, 

In [19]:
for batch in loader:
    print("Batch shape:", batch.shape)  # (batch_size, seq_len)

    # One-hot sur ce batch
    tensor_one_hot_clean = F.one_hot(batch, num_classes=len(vocab_clean))
    print("One-hot shape:", tensor_one_hot_clean.shape)



Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape: torch.Size([20, 66])
One-hot shape: torch.Size([20, 66, 3060])
Batch shape:

In [23]:
print(tensor_code_phrases_clean[6])

tensor([41, 42, 50, 51,  1, 51, 52,  1, 53, 20, 41, 54, 55, 24, 56, 57, 58,  2,
        59,  4, 60, 10, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
